### 1098. Unpopular Books(Median)



Table: Books
```
+----------------+---------+
| Column Name    | Type    |
+----------------+---------+
| book_id        | int     |
| name           | varchar |
| available_from | date    |
+----------------+---------+
```
book_id is the primary key of this table.
 

Table: Orders
```
+----------------+---------+
| Column Name    | Type    |
+----------------+---------+
| order_id       | int     |
| book_id        | int     |
| quantity       | int     |
| dispatch_date  | date    |
+----------------+---------+
```
order_id is the primary key of this table.
book_id is a foreign key to the Books table.

```
    select 
    book_id, name  
    from
    (
        select 
            b.book_id
            , b.name
            , case when datediff(b.available_from, '2019-06-23') < -30 then 1 else 0 end as is_valid
            , sum(case when datediff(o.dispatch_date, '2019-06-23') >= -365 then quantity else 0 end) over(partition by book_id) sum_copyies_valid
        from
            Books b
        left join
            Orders o
        on 
            b.book_id = o.book_id
    ) tmp
    where 
    tmp.is_valid = 1 and tmp.sum_copyies_valid < 10
    group by
    book_id, name  
    order by 
    book_id
```
  




### 1107. New Users Daily Count(Median)



Table: Traffic
```
+---------------+---------+
| Column Name   | Type    |
+---------------+---------+
| user_id       | int     |
| activity      | enum    |
| activity_date | date    |
+---------------+---------+
```
There is no primary key for this table, it may have duplicate rows.
The activity column is an ENUM type of ('login', 'logout', 'jobs', 'groups', 'homepage').
 

Write an SQL query to reports for every date within at most 90 days from today, the number of users that logged in for the first time on that date. Assume today is 2019-06-30.

Return the result table in any order.

The query result format is in the following example.

 

Example 1:

Input: 
Traffic table:
```
+---------+----------+---------------+
| user_id | activity | activity_date |
+---------+----------+---------------+
| 1       | login    | 2019-05-01    |
| 1       | homepage | 2019-05-01    |
| 1       | logout   | 2019-05-01    |
| 2       | login    | 2019-06-21    |
| 2       | logout   | 2019-06-21    |
| 3       | login    | 2019-01-01    |
| 3       | jobs     | 2019-01-01    |
| 3       | logout   | 2019-01-01    |
| 4       | login    | 2019-06-21    |
| 4       | groups   | 2019-06-21    |
| 4       | logout   | 2019-06-21    |
| 5       | login    | 2019-03-01    |
| 5       | logout   | 2019-03-01    |
| 5       | login    | 2019-06-21    |
| 5       | logout   | 2019-06-21    |
+---------+----------+---------------+
```
Output: 
```
+------------+-------------+
| login_date | user_count  |
+------------+-------------+
| 2019-05-01 | 1           |
| 2019-06-21 | 2           |
+------------+-------------+
```
Explanation: 
Note that we only care about dates with non zero user count.
The user with id 5 first logged in on 2019-03-01 so he's not counted on 2019-06-21.


```
    select
    activity_date as login_date, count(distinct(user_id)) as user_count
    from
    (
        select
        *
        , min(activity_date) over(partition by user_id) as min_activity_date
        from
        Traffic t
        where 
        activity = "login" 
    ) tmp
    where min_activity_date = activity_date
    group by 
    activity_date
    having 
    datediff(activity_date, '2019-06-30') >= -90
```

### 1112. Highest Grade For Each Student(Median)

Table: Enrollments
```
+---------------+---------+
| Column Name   | Type    |
+---------------+---------+
| student_id    | int     |
| course_id     | int     |
| grade         | int     |
+---------------+---------+
(student_id, course_id) is the primary key of this table.
``` 

Write a SQL query to find the highest grade with its corresponding course for each student. In case of a tie, you should find the course with the smallest course_id.

Return the result table ordered by student_id in ascending order.

The query result format is in the following example.


```
select student_id, min(course_id) as course_id, grade
from
(
    select 
        *, max(grade) over(partition by student_id) as max_score 
    from 
        Enrollments 
) tmp
where 
  tmp.grade = tmp.max_score
group by
  student_id, grade
```


### 1126. Active Businesses(Median)

Table: Events
```
+---------------+---------+
| Column Name   | Type    |
+---------------+---------+
| business_id   | int     |
| event_type    | varchar |
| occurences    | int     | 
+---------------+---------+
```
(business_id, event_type) is the primary key of this table.
Each row in the table logs the info that an event of some type occurred at some business for a number of times.
 

The average activity for a particular event_type is the average occurences across all companies that have this event.

An active business is a business that has more than one event_type such that their occurences is strictly greater than the average activity for that event.

Write an SQL query to find all active businesses.

Return the result table in any order.

The query result format is in the following example.

```
select
  business_id
from
(
    select
        *,
        avg(occurences) over(partition by event_type)  as avg_occur
    from 
        Events
)t1
where
  t1.occurences > t1.avg_occur
group by 
  business_id
having count(business_id) > 1

```

### 1127. User Purchase Platform(Hard)

```
+-------------+---------+
| Column Name | Type    |
+-------------+---------+
| user_id     | int     |
| spend_date  | date    |
| platform    | enum    | 
| amount      | int     |
+-------------+---------+
```
The table logs the history of the spending of users that make purchases from an online shopping website that has a desktop and a mobile application.
(user_id, spend_date, platform) is the primary key of this table.
The platform column is an ENUM type of ('desktop', 'mobile').
Write an SQL query to find the total number of users and the total amount spent using the mobile only, the desktop only, and both mobile and desktop together for each date.

Return the result table in any order.

The query result format is in the following example.

 

Example 1:

Input: 
Spending table:
```
+---------+------------+----------+--------+
| user_id | spend_date | platform | amount |
+---------+------------+----------+--------+
| 1       | 2019-07-01 | mobile   | 100    |
| 1       | 2019-07-01 | desktop  | 100    |
| 2       | 2019-07-01 | mobile   | 100    |
| 2       | 2019-07-02 | mobile   | 100    |
| 3       | 2019-07-01 | desktop  | 100    |
| 3       | 2019-07-02 | desktop  | 100    |
+---------+------------+----------+--------+
```
Output: 
```
+------------+----------+--------------+-------------+
| spend_date | platform | total_amount | total_users |
+------------+----------+--------------+-------------+
| 2019-07-01 | desktop  | 100          | 1           |
| 2019-07-01 | mobile   | 100          | 1           |
| 2019-07-01 | both     | 200          | 1           |
| 2019-07-02 | desktop  | 100          | 1           |
| 2019-07-02 | mobile   | 100          | 1           |
| 2019-07-02 | both     | 0            | 0           |
+------------+----------+--------------+-------------+ 
```
Explanation: 
On 2019-07-01, user 1 purchased using both desktop and mobile, user 2 purchased using mobile only and user 3 purchased using desktop only.
On 2019-07-02, user 2 purchased using mobile only, user 3 purchased using desktop only and no one purchased using both platforms.


```

with tmp1 as
(
    select 
    spend_date, 
    platform, 
    sum(amount) as total_amount, 
    count(user_id) as total_users
    from
    (
        select 
            spend_date, user_id,
            case when count(user_id) = 2 then 'both' else platform end as platform,
            case when count(user_id) = 2 then sum(amount) else amount end as amount
        from
            Spending s
        group by
            spend_date, user_id
    ) t1
    group by spend_date, platform
),

tmp2 as 
(
    select * from
    (
        select distinct(spend_date) from Spending
    ) t2
    cross join
    (	
        select 'mobile' as platform union
        select 'desktop' as platform union
        select "both" as platform 
    ) t1
)

select 
 tmp2.spend_date, tmp2.platform, 
 case when tmp1.total_amount <> 0 then tmp1.total_amount else 0 end as total_amount,
 case when tmp1.total_users <> 0 then tmp1.total_users else 0 end as total_users
from 
 tmp2 
left join 
 tmp1
on tmp1.spend_date = tmp2.spend_date and tmp1.platform = tmp2.platform



```

### 1158. Market Analysis I

Table: Users
```
+----------------+---------+
| Column Name    | Type    |
+----------------+---------+
| user_id        | int     |
| join_date      | date    |
| favorite_brand | varchar |
+----------------+---------+
```
user_id is the primary key of this table.
This table has the info of the users of an online shopping website where users can sell and buy items.
 

Table: Orders
```
+---------------+---------+
| Column Name   | Type    |
+---------------+---------+
| order_id      | int     |
| order_date    | date    |
| item_id       | int     |
| buyer_id      | int     |
| seller_id     | int     |
+---------------+---------+
```
order_id is the primary key of this table.
item_id is a foreign key to the Items table.
buyer_id and seller_id are foreign keys to the Users table.
 

Table: Items
```
+---------------+---------+
| Column Name   | Type    |
+---------------+---------+
| item_id       | int     |
| item_brand    | varchar |
+---------------+---------+
```
item_id is the primary key of this table.
 

Write an SQL query to find for each user, the join date and the number of orders they made as a buyer in 2019.

Return the result table in any order.

The query result format is in the following example.


```
with CTE as 
(
    select
    buyer_id, count(1) as orders_in_2019
    from
    Orders o
    where
    order_date >= '2019-01-01'
    and order_date < '2020-01-01'
    group by 
    buyer_id
)

select 
 u.user_id as buyer_id, u.join_date, case when c.orders_in_2019 is null then 0 else c.orders_in_2019 end as orders_in_2019
from
 Users u
left join
 CTE c
on
 u.user_id = c.buyer_id
```

### 1159. Market Analysis II

Table: Users
```
+----------------+---------+
| Column Name    | Type    |
+----------------+---------+
| user_id        | int     |
| join_date      | date    |
| favorite_brand | varchar |
+----------------+---------+
```
user_id is the primary key of this table.
This table has the info of the users of an online shopping website where users can sell and buy items.
 

Table: Orders
```
+---------------+---------+
| Column Name   | Type    |
+---------------+---------+
| order_id      | int     |
| order_date    | date    |
| item_id       | int     |
| buyer_id      | int     |
| seller_id     | int     |
+---------------+---------+
```
order_id is the primary key of this table.
item_id is a foreign key to the Items table.
buyer_id and seller_id are foreign keys to the Users table.
 

Table: Items
```
+---------------+---------+
| Column Name   | Type    |
+---------------+---------+
| item_id       | int     |
| item_brand    | varchar |
+---------------+---------+
```
item_id is the primary key of this table.
 

Write an SQL query to find for each user whether the brand of the second item (by date) they sold is their favorite brand. If a user sold less than two items, report the answer for that user as no. It is guaranteed that no seller sold more than one item on a day.

Return the result table in any order.

The query result format is in the following example.


```
with CTE as 
(
    select 
      seller_id, item_brand
    from
    (
        select 
        o.*, 
        i.item_brand,
        row_number() over(partition by seller_id order by order_date) as rn
        from
        Orders o
        join
        Items i
        on 
        o.item_id = i.item_id
    ) a
    where 
      rn = 2
)

select
  u.user_id as seller_id
  , case when c.item_brand = u.favorite_brand then 'yes' else 'no' end as 2nd_item_fav_brand
from
  Users u
left join
  CTE c
on
  u.user_id = c.seller_id
```